In [33]:
# B3 Data Visualization Dashboard

# Este notebook contém visualizações e análises dos dados B3 armazenados no AWS Glue Data Catalog.


In [34]:
# Instalação das bibliotecas necessárias
%pip install PyAthena pandas plotly boto3 sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Importação das bibliotecas
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [36]:
# Configurações de conexão (ajuste conforme necessário)
AWS_REGION = os.getenv('AWS_REGION', 'us-east-1')
S3_STAGING_DIR = os.getenv('S3_STAGING_DIR', 's3://861115334572-athena/b3-visualization/')
DATABASE = 'default'
TABLE = 'b3_tbl_refined'

In [37]:
# Consulta SQL para obter os dados necessários
df_query = f'''
SELECT code, reference_date, part, ticker
FROM {TABLE}
WHERE reference_date IS NOT NULL AND part IS NOT NULL AND ticker IS NOT NULL
AND code IN ('ITUB4', 'BPAC11', 'PETR4', 'VALE3', 'BBAS3')
ORDER BY reference_date, ticker
'''

In [ ]:
# Conexão com Athena
from sqlalchemy.engine import create_engine

conn_str = (
    f"awsathena+rest://@athena.{AWS_REGION}.amazonaws.com:443/{DATABASE}"
    f"?s3_staging_dir={S3_STAGING_DIR}"
)

conn = create_engine(conn_str)

In [39]:
# Executa a consulta e carrega em DataFrame
print('Executando consulta Athena...')
df = pd.read_sql(df_query, conn)

Executando consulta Athena...


In [40]:
# Converte reference_date para datetime
if not pd.api.types.is_datetime64_any_dtype(df['reference_date']):
    df['reference_date'] = pd.to_datetime(df['reference_date'])

print('Amostra dos dados:')
print(df.head())

Amostra dos dados:
     code reference_date    part        ticker
0   BBAS3     2025-06-20   2.889        BRASIL
1  BPAC11     2025-06-20   2.507    BTGP BANCO
2   ITUB4     2025-06-20   8.166  ITAUUNIBANCO
3   PETR4     2025-06-20   6.765     PETROBRAS
4   VALE3     2025-06-20  10.199          VALE


In [41]:
# Gráfico de linha: Evolução Temporal da Participação
fig = px.line(
    df,
    x='reference_date',
    y='part',
    color='ticker',
    title='Evolução Temporal da Participação por Ticker',
    labels={'reference_date': 'Data de Referência', 'part': 'Participação', 'ticker': 'Ticker'}
)
fig.update_layout(xaxis_title='Data', yaxis_title='Participação (%)')
fig.show() 

In [42]:
# --- DASHBOARD: Série temporal com bandas estatísticas ---
print("\n--- Dashboard: Série temporal com bandas estatísticas ---")


--- Dashboard: Série temporal com bandas estatísticas ---


In [43]:
# Consulta para médias e desvios
band_query = f'''
SELECT code, reference_date, ticker, mean_part_7_days, std_part_7_days
FROM {TABLE}
WHERE reference_date IS NOT NULL AND mean_part_7_days IS NOT NULL AND std_part_7_days IS NOT NULL AND ticker IS NOT NULL
AND code IN ('ITUB4', 'BPAC11', 'PETR4', 'VALE3', 'BBAS3')
ORDER BY reference_date, ticker
'''

band_df = pd.read_sql(band_query, conn)

In [44]:
# Converte reference_date para datetime
if not pd.api.types.is_datetime64_any_dtype(band_df['reference_date']):
    band_df['reference_date'] = pd.to_datetime(band_df['reference_date'])

In [45]:
# Calcula limites superior e inferior
band_df['upper'] = band_df['mean_part_7_days'] + band_df['std_part_7_days']
band_df['lower'] = band_df['mean_part_7_days'] - band_df['std_part_7_days']


In [46]:
# Gráfico com plotly: linha + banda sombreada
tickers = band_df['ticker'].unique()
for ticker in tickers:
    df_ticker = band_df[band_df['ticker'] == ticker]
    fig = go.Figure()
    # Banda sombreada
    fig.add_traces([
        go.Scatter(
            x=pd.concat([df_ticker['reference_date'], df_ticker['reference_date'][::-1]]),
            y=pd.concat([df_ticker['upper'], df_ticker['lower'][::-1]]),
            fill='toself',
            fillcolor='rgba(0,100,80,0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            showlegend=False,
            name='Intervalo de Confiança'
        ),
        go.Scatter(
            x=df_ticker['reference_date'],
            y=df_ticker['mean_part_7_days'],
            line=dict(color='rgb(0,100,80)'),
            name=f'Média 7 dias ({ticker})'
        )
    ])
    fig.update_layout(
        title=f'Série Temporal com Bandas Estatísticas - {ticker}',
        xaxis_title='Data',
        yaxis_title='Participação Média (7 dias)',
        legend_title='Legenda'
    )
    fig.show() 

In [47]:

# --- Boxplot de part por ticker ---
print("\n--- Dashboard: Boxplot de part por ticker ---")

boxplot_query = f'''
SELECT part, ticker, code
FROM {TABLE}
WHERE part IS NOT NULL AND ticker IS NOT NULL
AND cast(reference_date as date) = date_add('day', -1, current_date)
'''

boxplot_df = pd.read_sql(boxplot_query, conn)

# Gráfico boxplot com plotly
fig = px.box(
    boxplot_df,
    x='ticker',
    y='part',
    title='Distribuição da Participação (part) por Ticker',
    labels={'ticker': 'Ticker', 'part': 'Participação'}
)
fig.update_layout(xaxis_title='Ticker', yaxis_title='Participação (%)')
fig.show() 


--- Dashboard: Boxplot de part por ticker ---


In [48]:
# --- Dispersão mean_part_7_days vs std_part_7_days ---
print("\n--- Dashboard: Dispersão mean_part_7_days vs std_part_7_days ---")

scatter_query = f'''
SELECT mean_part_7_days, std_part_7_days, ticker
FROM {TABLE}
WHERE mean_part_7_days IS NOT NULL AND std_part_7_days IS NOT NULL AND ticker IS NOT NULL
AND cast(reference_date as date) = date_add('day', -1, current_date)
'''

scatter_df = pd.read_sql(scatter_query, conn)

fig = px.scatter(
    scatter_df,
    x='mean_part_7_days',
    y='std_part_7_days',
    color='ticker',
    title='Dispersão: Média vs Volatilidade da Participação (por Ticker)',
    labels={'mean_part_7_days': 'Média 7 dias', 'std_part_7_days': 'Desvio Padrão 7 dias', 'ticker': 'Ticker'},
    hover_data=['ticker']
)
fig.update_layout(xaxis_title='Participação Média (7 dias)', yaxis_title='Volatilidade (Desvio Padrão 7 dias)')
fig.show()


--- Dashboard: Dispersão mean_part_7_days vs std_part_7_days ---
